In [ ]:
import numpy as np
import pandas as pd
from pandas import concat
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, r2_score
from sklearn.preprocessing import MinMaxScaler
from numpy import concatenate
from math import sqrt
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

In [ ]:
#Read dataset
df = pd.read_csv("")   

In [ ]:
import numpy as np

# --------- SELECT COLUMNS ----------
features = [
    'accx', 'accy', 'accz',
    'eda_f', 'temp_f', 'HR'
]
target = 'In_solving_or_studying_the_topics_in_lecture_I_invested'

# --------- BUILD DATASET ----------
dataset = df[features + [target]].to_numpy()

# --------- SEQUENCE SPLITTING ----------
def split_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i+n_steps, :-1])
        y.append(data[i+n_steps, -1])
    return np.array(X), np.array(y)

# --------- PARAMETERS ----------
n_steps = 10

# --------- CREATE INPUT / OUTPUT ----------
X, y = split_sequences(dataset, n_steps)

print(X.shape, y.shape)

In [ ]:
def run_cnn_split(X,y, train_ratio):
    # Initialize and fit scaler for the target variable 'y'
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Reshape y to 2D for scaling, then scale it
    y_scaled = scaler.fit_transform(y.reshape(-1, 1))
    n_train = int(len(X) * train_ratio)
    X_train = X[:n_train, :]
    Y_train = y_scaled[:n_train]
    test_X = X[n_train:, :]
    test_y = y_scaled[n_train:]

    n_features = X.shape[2]

    # build model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps, n_features)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, Y_train, epochs=10, verbose=0)

    train_yhat_scaled = model.predict(X_train)
    inv_train_yhat = scaler.inverse_transform(train_yhat_scaled)
    inv_train_y = scaler.inverse_transform(Y_train)

    train_rmse = np.sqrt(mean_squared_error(inv_train_y, inv_train_yhat))
    train_f1 = f1_score(np.rint(inv_train_y), np.rint(inv_train_yhat),average='weighted')
    train_r2   = r2_score(inv_train_y, inv_train_yhat)

    # ---------- TEST ----------
    test_yhat_scaled = model.predict(test_X)

    inv_test_yhat = scaler.inverse_transform(test_yhat_scaled)
    inv_test_y = scaler.inverse_transform(test_y)

    test_rmse = np.sqrt(mean_squared_error(inv_test_y, inv_test_yhat))
    test_f1 = f1_score(np.rint(inv_test_y), np.rint(inv_test_yhat),average='weighted')
    test_r2   = r2_score(inv_test_y, inv_test_yhat)

    # transitions (on predicted TEST signal)
    test_present_trans = (np.diff(inv_test_y.flatten())!=0).sum()
    x_roun = np.round(inv_test_yhat.flatten())
    test_detect_trans=(np.diff(x_roun)!=0).sum()
    
    train_pct = round(train_ratio * 100)
    test_pct  = 100 - train_pct
    
    return {
        "Train %": train_pct,
        "Test %": test_pct,
        "Train RMSE": train_rmse,
        "Test RMSE": test_rmse,
        "Train R2": train_r2,
        "Test R2": test_r2,
        "Train F1": train_f1,
        "Test F1": test_f1,
        "Transitions present in Test Set": test_present_trans,
        "Transitions Detected in Test Set": test_detect_trans
    }

In [ ]:
results_cnn = []

for r in np.arange(0.1, 1.0, 0.1):
    res = run_cnn_split(X,y, train_ratio=r)
    results_cnn.append(res)

results_df_cnn = pd.DataFrame(results_cnn)
results_df_cnn